<center>
    <img src="https://gitlab.com/ibm/skills-network/courses/placeholder101/-/raw/master/labs/module%201/images/IDSNlogo.png"  width="300" alt="cognitiveclass.ai logo"  />
</center>

<h1 align=center><font size = 5>Assignment: SQL Notebook for Peer Assignment</font></h1>

Estimated time needed: **60** minutes.

## Introduction

Using this Python notebook you will:

1.  Understand the Spacex DataSet
2.  Load the dataset  into the corresponding table in a Db2 database
3.  Execute SQL queries to answer assignment questions

## Overview of the DataSet

SpaceX has gained worldwide attention for a series of historic milestones.

It is the only private company ever to return a spacecraft from low-earth orbit, which it first accomplished in December 2010.
SpaceX advertises Falcon 9 rocket launches on its website with a cost of 62 million dollars wheras other providers cost upward of 165 million dollars each, much of the savings is because Space X can reuse the first stage.

Therefore if we can determine if the first stage will land, we can determine the cost of a launch.

This information can be used if an alternate company wants to bid against SpaceX for a rocket launch.

This dataset includes a record for each payload carried during a SpaceX mission into outer space.

In [50]:
import pandas as pd
import sqlite3

!pip install ipython-sql

In [4]:
spacex = pd.read_csv('spacex.csv')
spacex.head()

,Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing _Outcome
0,04-06-2010,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
1,08-12-2010,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of...",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2,22-05-2012,07:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
3,08-10-2012,00:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
4,01-03-2013,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


In [7]:
spacex.rename(columns={'PAYLOAD_MASS__KG_': 'Payload_mass_kg', 'Landing _Outcome': 'Landing_outcome'}, inplace =True)

In [11]:
cnn = sqlite3.connect('spacex_falcon9.db')

In [12]:
spacex.to_sql('spacex', cnn)

In [13]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [15]:
%sql sqlite:///spacex_falcon9.db

### Task 1

##### Display the names of the unique launch sites  in the space mission

In [18]:
%%sql
select distinct (Launch_Site) from spacex

   sqlite:///jupyter_sql
 * sqlite:///spacex_falcon9.db
Done.


Launch_Site
CCAFS LC-40
VAFB SLC-4E
KSC LC-39A
CCAFS SLC-40


### Task 2

##### Display the Five records where launch sites begin with `CCA`

In [22]:
%%sql 
SELECT Launch_site from spacex where Launch_site LIKE 'CCA%' LIMIT 5;

   sqlite:///jupyter_sql
 * sqlite:///spacex_falcon9.db
Done.


Launch_Site
CCAFS LC-40
CCAFS LC-40
CCAFS LC-40
CCAFS LC-40
CCAFS LC-40


### Task 3
##### Display the total payload mass carried by boosters launched by NASA (CRS)

In [26]:
%%sql 

select sum(Payload_mass_kg) as Payload_mass from spacex;

   sqlite:///jupyter_sql
 * sqlite:///spacex_falcon9.db
Done.


Payload_mass
619967


### Task 4

##### Display the average payload mass carried by booster version F9 v1.1 

In [28]:
%%sql 

select avg(Payload_mass_kg) as Payload_mass from spacex where Booster_Version = 'F9 v1.1' 

   sqlite:///jupyter_sql
 * sqlite:///spacex_falcon9.db
Done.


Payload_mass
2928.4


### Task 5

##### Display successful landing outcome on ground pad

In [29]:
%%sql

SELECT Date, Landing_outcome FROM spacex WHERE Landing_outcome = 'Success (ground pad)'

   sqlite:///jupyter_sql
 * sqlite:///spacex_falcon9.db
Done.


Date,landing_outcome
22-12-2015,Success (ground pad)
18-07-2016,Success (ground pad)
19-02-2017,Success (ground pad)
01-05-2017,Success (ground pad)
03-06-2017,Success (ground pad)
14-08-2017,Success (ground pad)
07-09-2017,Success (ground pad)
15-12-2017,Success (ground pad)
08-01-2018,Success (ground pad)


### Task 6

##### List the names of the boosters which have success in drone ship and have payload mass greater than 4000 but less than 6000

In [30]:
%%sql

SELECT Booster_Version, Payload_mass_kg, Landing_outcome FROM spacex WHERE Landing_outcome = 'Success (drone ship)' AND
Payload_mass_kg BETWEEN 4000 and 6000

   sqlite:///jupyter_sql
 * sqlite:///spacex_falcon9.db
Done.


Booster_Version,payload_mass_kg,landing_outcome
F9 FT B1022,4696,Success (drone ship)
F9 FT B1026,4600,Success (drone ship)
F9 FT B1021.2,5300,Success (drone ship)
F9 FT B1031.2,5200,Success (drone ship)


### Task 7

##### List the total number of successful and failure mission outcomes

In [41]:
%%sql

SELECT Mission_Outcome, COUNT(Mission_Outcome) FROM spacex 
GROUP BY Mission_Outcome;

   sqlite:///jupyter_sql
 * sqlite:///spacex_falcon9.db
Done.


Mission_Outcome,COUNT(Mission_Outcome)
Failure (in flight),1
Success,98
Success,1
Success (payload status unclear),1


### Task 8

##### List the   names of the booster_versions which have carried the maximum payload mass. Use a subquery

In [42]:
%%sql

SELECT Booster_Version, MAX(Payload_mass_kg) FROM spacex;

   sqlite:///jupyter_sql
 * sqlite:///spacex_falcon9.db
Done.


Booster_Version,MAX(Payload_mass_kg)
F9 B5 B1048.4,15600


### Task 9

##### List the failed landing_outcomes in drone ship, their booster versions, and launch site names for in year 2015

In [46]:
%%sql

SELECT Date, Booster_Version, Launch_Site, Landing_Outcome FROM spacex WHERE Landing_Outcome = 'Failure (drone ship)'
AND Date LIKE '%____2015'

   sqlite:///jupyter_sql
 * sqlite:///spacex_falcon9.db
Done.


Date,Booster_Version,Launch_Site,landing_outcome
10-01-2015,F9 v1.1 B1012,CCAFS LC-40,Failure (drone ship)
14-04-2015,F9 v1.1 B1015,CCAFS LC-40,Failure (drone ship)


### Task 10

##### Rank the count of landing outcomes (such as Failure (drone ship) or Success (ground pad)) between the date 2010-06-04 and 2017-03-20, in descending order


In [47]:
%%sql

SELECT Date, Landing_outcome, COUNT(Landing_outcome) FROM spacex WHERE Date BETWEEN '04-06-2010' AND '20-03-2017'
ORDER BY COUNT(Landing_outcome) DESC

   sqlite:///jupyter_sql
 * sqlite:///spacex_falcon9.db
Done.


Date,landing_outcome,COUNT(Landing_outcome)
04-06-2010,Failure (parachute),57
